<a href="https://colab.research.google.com/github/doumoh/RIS_aided_communication/blob/main/Neural_Receiver_for_OFDM_SIMO___.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall -y sionna tensorflow tensorflow-probability numpy
!pip install sionna==0.19 tensorflow tensorflow-probability numpy --upgrade
!pip uninstall -y mitsuba
!pip install mitsuba==3.5.0

In [153]:
import os
if os.getenv("CUDA_VISIBLE_DEVICES") is None:
    gpu_num = 0 # Use "" to use the CPU
    os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Import Sionna
try:
    import sionna
except ImportError as e:
    # Install Sionna if package is not already installed
    import os
    os.system("pip install sionna==0.19")
    import sionna

# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

sionna.config.seed = 42 # Set seed for reproducible random number generation
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pickle

from tensorflow.keras import Model
from tensorflow.keras.layers import Layer, Conv2D, LayerNormalization
from tensorflow.nn import relu

from sionna.channel.tr38901 import Antenna, AntennaArray
from sionna.channel import OFDMChannel
from sionna.mimo import StreamManagement
from sionna.ofdm import ResourceGrid, ResourceGridMapper, LSChannelEstimator, LMMSEEqualizer, RemoveNulledSubcarriers, ResourceGridDemapper
from sionna.utils import BinarySource, ebnodb2no, insert_dims, flatten_last_dims, log10, expand_to_rank
from sionna.fec.ldpc.encoding import LDPC5GEncoder
from sionna.fec.ldpc.decoding import LDPC5GDecoder
from sionna.mapping import Mapper, Demapper
from sionna.utils.metrics import compute_ber
from sionna.utils import sim_ber
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, Camera
from sionna.channel import cir_to_ofdm_channel, subcarrier_frequencies, OFDMChannel, ApplyOFDMChannel, CIRDataset
from sionna.nr import PUSCHConfig, PUSCHTransmitter, PUSCHReceiver
from sionna.utils import compute_ber, ebnodb2no, PlotBER
from sionna.mimo import StreamManagement
from sionna.rt import load_scene, Transmitter, Receiver, RIS, PlanarArray, normalize, Camera
from sionna import PI
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/Blender Scene/rxx.xml'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [154]:
# Load scene
scene = load_scene(file_path)
# Configure antenna array for all transmitters
scene.tx_array = PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="dipole",
                             polarization="H")

# Configure antenna array for all receivers
scene.rx_array = PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="dipole",
                             polarization="cross")

# Create transmitter
tx = Transmitter(name="tx",
                 position=[-4,3,3])

# Add transmitter
scene.add(tx)

width = 8 * scene.wavelength # Width [m] for 16*16 RIS elements
num_rows = num_cols = int(width/(0.5*scene.wavelength))
ris = RIS(name="ris",
          position=[7.5,-3,2],
          orientation=[PI/2,0,0],
          num_rows=num_rows,
          num_cols=num_cols)

scene.add(ris)

# Create a receiver
rx = Receiver(name="rx",
              position=[3,4.5,3.5],
              orientation=[0,0,0])

# Add receiver
scene.add(rx)


In [155]:
from sionna.rt.solver_paths import SolverPaths # Import SolverPaths to modify the _ris_transition_matrices function
class CustomSolverPaths(SolverPaths):
    def _ris_transition_matrices(self, ris_paths, ris_paths_tmp):
        # Compute scattering coefficients
        sc = [tf.reduce_sum(r(), axis=0) for r in self._scene.ris.values()]
        sc = tf.concat(sc, axis=0)
        sc = sc[tf.newaxis, tf.newaxis, ...]


        # Coefficient calculation
        coef = tf.cast(4 * PI, self._rdtype)
        coef /= tf.reduce_prod(ris_paths_tmp.distances, axis=0)
        coef *= tf.cast(tf.sqrt(tx.power_dbm * 0.001 * 120 * PI * 2 * 66), self._rdtype)
        coef = tf.complex(coef, tf.cast(0, self._rdtype))

        # Differentiable phase decision: use magnitude of imag part
        imag_part = tf.math.imag(sc)

        # Sharpen sigmoid to approximate a step function: large scaling factor
        sharpness = 100000000.0  # Higher value = closer to hard decision
        phase_weight = tf.sigmoid(sharpness * imag_part)  # Step at imag_part = 0

        # Now interpolate between 0° for real (imag == 0) and 165° for complex (non-zero imag)
        phase_deg = phase_weight * 2 * 165.0


        # Convert to radians and cast
        phase_rad = phase_deg * (np.pi / 180.0)
        phase_rad = tf.cast(phase_rad, self._rdtype)


        # Compute complex phase rotation
        sigma_eff_value = 0.001  # Scalar value
        sqrt_sigma = tf.sqrt(tf.cast(sigma_eff_value, self._rdtype))
        sqrt_sigma = tf.complex(sqrt_sigma, tf.cast(0, self._rdtype))  # Ensure sqrt_sigma is complex
        sigma_phi_matrix = sqrt_sigma * tf.exp(
            tf.complex(0.0, phase_rad)  # Ensure exp result is complex
        )

        # Apply coefficient modification with masking
        coef *= sigma_phi_matrix
        coef = tf.where(ris_paths.mask, coef, tf.cast(0, coef.dtype))

        # Create polarization-preserving transition matrices
        coef = coef[..., tf.newaxis, tf.newaxis]
        ris_mat_t = coef * tf.eye(2, batch_shape=[1, 1, 1], dtype=self._dtype)

        return ris_mat_t
scene._solver_paths = CustomSolverPaths(scene) # Replace to custom SolverPaths


In [156]:
# SNR range for evaluation and training
ebno_db_min = -5.0
ebno_db_max = 10.0

## OFDM configuration
subcarrier_spacing = 30e3 # Hz
fft_size = 128 # Number of subcarriers
num_ofdm_symbols = 14 # Number of OFDM symbols
dc_null = True # Null the DC subcarrier
num_guard_carriers = [5, 6] # Number of guard carriers on each side
pilot_pattern = "kronecker" # Pilot pattern
pilot_ofdm_symbol_indices = [2, 11] # Index of OFDM symbols carrying pilots
#pilot_ofdm_symbol_indices = [2] # Index of OFDM symbols carrying pilots
cyclic_prefix_length = 0 # Simulation in frequency domain.

## Modulation and coding configuration
num_bits_per_symbol = 2 # QPSK
coderate = 0.5 # Coderate for LDPC code

## Neural receiver configuration
num_conv_channels = 12 # Number of convolutional channels for the convolutional layers forming the neural receiver

## Training configuration
training_batch_size = 64 # Training batch size
model_weights_path = "neural_receiver_weights" # Location to save the neural receiver weights once training is done
stream_manager = StreamManagement(np.array([[1]]), # Receiver-transmitter association matrix
                                  1)               # One stream per transmitter
resource_grid = ResourceGrid(num_ofdm_symbols = num_ofdm_symbols,
                             fft_size = fft_size,
                             subcarrier_spacing = subcarrier_spacing,
                             num_tx = 1,
                             num_streams_per_tx = 1,
                             cyclic_prefix_length = cyclic_prefix_length,
                             dc_null = dc_null,
                             pilot_pattern = pilot_pattern,
                             pilot_ofdm_symbol_indices = pilot_ofdm_symbol_indices,
                             num_guard_carriers = num_guard_carriers)
# Codeword length. It is calculated from the total number of databits carried by the resource grid, and the number of bits transmitted per resource element
n = int(resource_grid.num_data_symbols*num_bits_per_symbol)
# Number of information bits per codeword
k = int(n*coderate)

In [157]:
paths = scene.compute_paths(max_depth=5,
                            num_samples=1e6)

a, tau = paths.cir()
# Compute frequencies of subcarriers and center around carrier frequency
frequencies = subcarrier_frequencies(fft_size, subcarrier_spacing)

# Compute the frequency response of the channel at frequencies.
h_freq = cir_to_ofdm_channel(frequencies,
                             a,
                             tau,
                             normalize=True)

In [175]:
binary_source = BinarySource()
mapper = Mapper("qam", num_bits_per_symbol)
rg_mapper = ResourceGridMapper(resource_grid)
batch_size = 64
ebno_db = tf.fill([batch_size], 25.0)
no = ebnodb2no(ebno_db, num_bits_per_symbol, coderate)
c = binary_source([batch_size, 1, 1, n])
x = mapper(c)
x_rg = rg_mapper(x)
channel = ApplyOFDMChannel(add_awgn=True)

In [ ]:
def clean_subcarriers(ofdm_symbol, dc_null=True, guard_left=5, guard_right=6):
    # Remove guards
    active = ofdm_symbol[guard_left:-guard_right]
    if dc_null:
        # Remove DC (middle subcarrier)
        dc_idx = (len(active)) // 2
        active = tf.concat([active[:dc_idx], active[dc_idx+1:]], axis=0)
    return active
# Keep track of best loss and corresponding phase configuration
best_loss = np.inf
# Create trainable variables for phase (continuous)
phase_var = tf.Variable(tf.zeros_like(ris.phase_profile.values), trainable=True)
best_phase_config = tf.Variable(tf.zeros_like(ris.phase_profile.values), trainable=False)
optimizer = tf.keras.optimizers.Adam()
@tf.custom_gradient
def binarize_phase(x):
    pi = tf.constant(np.pi, dtype=tf.float32)
    binary = tf.where(x > 0, pi, 0.0)         # Anything > 0 --> pi else 0
    def grad(dy):
        return dy * tf.cast(tf.logical_and(x > -1.0, x < 1.0), tf.float32) # Accept gradient between -1.5 and 1.5 ,other set to zero (no update)
    return binary, grad

def to_db(x):
    return 10*tf.math.log(x)/tf.math.log(10.)
# Training step
def train_step():
    global best_loss, best_phase_config

    with tf.GradientTape() as tape:
        # Apply binarized and differentiable phase
        bin_phase = binarize_phase(phase_var)

        # Apply binarized phase to RIS
        ris.phase_profile.values = bin_phase

        # Compute channel response and output
        paths = scene.compute_paths(max_depth=5, num_samples=1e6)
        a, tau = paths.cir()
        frequencies = subcarrier_frequencies(fft_size, subcarrier_spacing)
        h_freq = cir_to_ofdm_channel(frequencies, a, tau, normalize=True)
        y = channel([x_rg, h_freq, no])

        # Extract pilots and compute loss
        y_pilots_sym2 = y[0, 0, 0, 2]  # shape: (128,)
        active_sym2 = clean_subcarriers(y_pilots_sym2)
        energy_per_symbol2 = tf.abs(active_sym2) ** 2
        loss1 = tf.reduce_sum(energy_per_symbol2)
        print(loss1)
        # Compute loss on pilot energy
        y_pilots_sym11 = y[0, 0, 0, 11]
        active_sym11 = clean_subcarriers(y_pilots_sym11)
        energy_per_symbol11 = tf.abs(active_sym11) ** 2
        loss2 = tf.reduce_sum(energy_per_symbol11)
        print(loss2)
        loss = (loss1 + loss2)

    # Check if this is a better phase config (i.e., lower loss)
    if loss < best_loss:
        best_loss = loss
        best_phase_config.assign(phase_var)  # Save current phase_var
       # ris.phase_profile.values = phase_var
        grads = tape.gradient(loss, [phase_var])
        optimizer.apply_gradients(zip(grads, [phase_var]))
        print(f" Accepted update: Loss = {loss.numpy():.2f}")
    else:
        phase_var.assign(best_phase_config)  # Revert to best
        print(f" Rejected update: Loss = {loss.numpy():.2f} (Best = {best_loss.numpy():.2f})")

    return loss
num_iterations = 15
for i in range(num_iterations):
    loss = train_step()
    print(f"Iteration {i}: Loss = {loss.numpy():.2f} ")


In [160]:
class ResidualBlock(Layer):

    def build(self, input_shape):

        # Layer normalization is done over the last three dimensions: time, frequency, conv 'channels'
        self._layer_norm_1 = LayerNormalization(axis=(-1, -2, -3))
        self._conv_1 = Conv2D(filters=num_conv_channels,
                              kernel_size=[2,2],
                              padding='same',
                              activation=None)
        # Layer normalization is done over the last three dimensions: time, frequency, conv 'channels'
        self._layer_norm_2 = LayerNormalization(axis=(-1, -2, -3))
        self._conv_2 = Conv2D(filters=num_conv_channels,
                              kernel_size=[2,2],
                              padding='same',
                              activation=None)

    def call(self, inputs):
        z = self._layer_norm_1(inputs)
        z = relu(z)
        z = self._conv_1(z)
        z = self._layer_norm_2(z)
        z = relu(z)
        z = self._conv_2(z)
        z = z + inputs

        return z

class NeuralReceiver(Layer):

    def build(self, input_shape):

        # Input convolution
        self._input_conv = Conv2D(filters=num_conv_channels,
                                  kernel_size=[2,2],
                                  padding='same',
                                  activation=None)
        # Residual blocks
        self._res_block_1 = ResidualBlock()
        self._res_block_2 = ResidualBlock()
        self._res_block_3 = ResidualBlock()
        self._res_block_4 = ResidualBlock()
        # Output conv
        self._output_conv = Conv2D(filters=num_bits_per_symbol,
                                   kernel_size=[2,2],
                                   padding='same',
                                   activation=None)

    def call(self, inputs):
        y, no = inputs

        # Feeding the noise power in log10 scale
        no = log10(no)
        # Stacking the real and imaginary components of the different antennas along the 'channel' dimension
        y = tf.transpose(y, [0, 2, 3, 1]) # Putting antenna dimension last
        no = insert_dims(no, 3, 1)
        no = tf.tile(no, [1, y.shape[1], y.shape[2], 1])
        z = tf.concat([tf.math.real(y),
                       tf.math.imag(y),
                       no], axis=-1)
        # Input conv
        z = self._input_conv(z)
        # Residual blocks
        z = self._res_block_1(z)
        z = self._res_block_2(z)
        z = self._res_block_3(z)
        z = self._res_block_4(z)
        # Output conv
        z = self._output_conv(z)

        return z

In [161]:
class E2ESystem(Model):


    def __init__(self, system, training=False):
        super().__init__()
        self._system = system
        self._training = training

        ######################################
        ## Transmitter
        self._binary_source = BinarySource()
        if not training:
            self._encoder = LDPC5GEncoder(k, n)
        self._mapper = Mapper("qam", num_bits_per_symbol)
        self._rg_mapper = ResourceGridMapper(resource_grid)
        ######################################
        ## Channel
        self._channel = ApplyOFDMChannel(add_awgn=True)
        ######################################
        ## Receiver
        if "baseline" in system:
            if system == 'baseline-perfect-csi':  # Perfect CSI
                self._removed_null_subc = RemoveNulledSubcarriers(resource_grid)
            elif system == 'baseline-ls-estimation':  # LS estimation
                self._ls_est = LSChannelEstimator(resource_grid, interpolation_type="nn")
            # Components required by both baselines
            self._lmmse_equ = LMMSEEqualizer(resource_grid, stream_manager)
            self._demapper = Demapper("app", "qam", num_bits_per_symbol)

        elif system == "neural-receiver":  # Neural receiver
            self._neural_receiver = NeuralReceiver()
            self._rg_demapper = ResourceGridDemapper(resource_grid, stream_manager)  # Used to extract data-carrying resource elements
        if not training:
            self._decoder = LDPC5GDecoder(self._encoder, hard_out=True)
    @tf.function
    def call(self, batch_size, ebno_db):

        # If `ebno_db` is a scalar, a tensor with shape [batch size] is created as it is what is expected by some layers
        if len(ebno_db.shape) == 0:
            ebno_db = tf.fill([batch_size], ebno_db)
        no = ebnodb2no(ebno_db, num_bits_per_symbol, coderate)
        if self._training:
            c = self._binary_source([batch_size, 1, 1, n])
        else:
            b = self._binary_source([batch_size, 1, 1, k])
            c = self._encoder(b)
        # Modulation
        x = self._mapper(c)
        x_rg = self._rg_mapper(x)
        no_ = expand_to_rank(no, tf.rank(x_rg))
        # channel
        y   = self._channel([x_rg,h_freq, no_])
        if "baseline" in self._system:
            if self._system == 'baseline-perfect-csi':
                h_hat = self._removed_null_subc(h_freq) # Extract non-null subcarriers
                batch_size = tf.shape(y)[0]  # Or pass batch_size explicitly if available
                # Tile h_hat across the batch dimension
                h_hat = tf.tile(h_hat, [batch_size, 1, 1, 1, 1, 14, 1])
                err_var = 0.0 # No channel estimation error when perfect CSI knowledge is assumed
            elif self._system == 'baseline-ls-estimation':
                h_hat, err_var = self._ls_est([y, no]) # LS channel estimation with nearest-neighbor


            x_hat, no_eff = self._lmmse_equ([y, h_hat, err_var, no]) # LMMSE equalization
            no_eff_= expand_to_rank(no_eff, tf.rank(x_hat))
            llr = self._demapper([x_hat, no_eff_]) # Demapping
        elif self._system == "neural-receiver":
            # The neural receiver computes LLRs from the frequency domain received symbols and N0
            y = tf.squeeze(y, axis=1)
            llr = self._neural_receiver([y, no])
            llr = insert_dims(llr, 2, 1) # Reshape the input to fit what the resource grid demapper is expected
            llr = self._rg_demapper(llr) # Extract data-carrying resource elements. The other LLrs are discarded
            llr = tf.reshape(llr, [batch_size, 1, 1, n]) # Reshape the LLRs to fit what the outer decoder is expected
        if self._training:
            # Compute and return BMD rate (in bit), which is known to be an achievable
            # information rate for BICM systems.
            # Training aims at maximizing the BMD rate
            bce = tf.nn.sigmoid_cross_entropy_with_logits(c, llr)
            bce = tf.reduce_mean(bce)
            rate = tf.constant(1.0, tf.float32) - bce/tf.math.log(2.)
            return rate
        else:
            # Outer decoding
            b_hat = self._decoder(llr)
            return b,b_hat # for BER/BLER computation


In [ ]:
num_training_iterations = 300 # Number of training iterations
training = True
if training:
    model = E2ESystem('neural-receiver', training=True)

    optimizer = tf.keras.optimizers.Adam()

    for i in range(num_training_iterations):
        # Sampling a batch of SNRs
        ebno_db = tf.random.uniform(shape=[], minval=ebno_db_min, maxval=ebno_db_max)
        # Forward pass
        with tf.GradientTape() as tape:
            rate = model(training_batch_size, ebno_db)
            loss = - rate
            print(loss.numpy())

        # Computing and applying gradients
        weights = model.trainable_weights
        grads = tape.gradient(loss, weights)
        optimizer.apply_gradients(zip(grads, weights))
        if i % 100 == 0:
            print('Iteration {}/{}  Rate: {:.4f} bit'.format(i, num_training_iterations, rate.numpy()), end='\r')



In [ ]:
def save_weights(model, model_weights_path):
    weights = model.get_weights()
    with open(model_weights_path, 'wb') as f:
        pickle.dump(weights, f)
save_weights(model, model_weights_path)
# Range of SNRs over which the systems are evaluated
ebno_dbs = np.arange(ebno_db_min, # Min SNR for evaluation
                     ebno_db_max, # Max SNR for evaluation
                     1) # Step
# function to load and set weights of a model
def load_weights(model, model_weights_path):
    model(1, tf.constant(10.0, tf.float32))
    with open(model_weights_path, 'rb') as f:
        weights = pickle.load(f)
    model.set_weights(weights)
# Dictionary storing the evaluation results
BER = {}

model = E2ESystem('baseline-perfect-csi')
ber,_ = sim_ber(model, ebno_dbs, batch_size=64, num_target_block_errors=100, max_mc_iter=100)
BER['baseline-perfect-csi-OpRIS'] = ber.numpy()

model = E2ESystem('baseline-ls-estimation')
ber,_ = sim_ber(model, ebno_dbs, batch_size=64, num_target_block_errors=100, max_mc_iter=100)
BER['baseline-ls-estimation-OpRIS'] = ber.numpy()

model = E2ESystem('neural-receiver')

# Run one inference to build the layers and loading the weights
model(1, tf.constant(10.0, tf.float32))
with open(model_weights_path, 'rb') as f:
    weights = pickle.load(f)
model.set_weights(weights)

# Evaluations
ber,_ = sim_ber(model, ebno_dbs, batch_size=64, num_target_block_errors=100, max_mc_iter=100)
BER['neural-receiver-OpRIS'] = ber.numpy()

In [ ]:
binary_source = BinarySource()
mapper = Mapper("qam", num_bits_per_symbol)
rg_mapper = ResourceGridMapper(resource_grid)
#neural_receiver = model_conventional._neural_receiver
rg_demapper = ResourceGridDemapper(resource_grid, stream_manager)
channel = ApplyOFDMChannel(add_awgn=True)
batch_size = 64
ebno_db = tf.fill([batch_size], 25.0)
no = ebnodb2no(ebno_db, num_bits_per_symbol, coderate)
## Transmitter
# Generate codewords
c = binary_source([batch_size, 1, 1, n])
print("c shape: ", c.shape)
# Map bits to QAM symbols
x = mapper(c)
print("x shape: ", x.shape)
# Map the QAM symbols to a resource grid
x_rg = rg_mapper(x)
print("x_rg shape: ", x_rg.shape)
print("h_freq shape: ", h_freq.shape)
######################################
## Channel
no_ = expand_to_rank(no, tf.rank(x_rg))
print("no shape: ", no.shape)
print("no_ shape: ", no_.shape)
# Apply channel
channel = ApplyOFDMChannel(add_awgn=True)
y = channel([x_rg, h_freq, no])
print("y shape: ", y.shape)
######################################
#_lmmse_equ = LMMSEEqualizer(resource_grid, stream_manager, )
##_removed_null_subc = RemoveNulledSubcarriers(resource_grid)
######################################
## Receiver
_lmmse_equ = LMMSEEqualizer(resource_grid, stream_manager)
_demapper = Demapper("app", "qam", num_bits_per_symbol)
_ls_est = LSChannelEstimator(resource_grid, interpolation_type="nn")
# The neural receiver computes LLRs from the frequency domain received symbols and N0
#y = tf.squeeze(y, axis=1)
#h_hat = _removed_null_subc(h_freq) # Extract non-null subcarriers
#batch_size = tf.shape(y)[0]  # Or pass batch_size explicitly if available
h_hat, err_var = _ls_est([y, no])
# Tile h_hat across the batch dimension
#h_hat = tf.tile(h_hat, [batch_size, 1, 1, 1, 1, 14, 1])
x_hat, no_eff = _lmmse_equ([y, h_hat, err_var, no]) # LMMSE equalization
no_eff_= expand_to_rank(no_eff, tf.rank(x_hat))
llr = _demapper([x_hat, no_eff_]) # Demapping


bce = tf.nn.sigmoid_cross_entropy_with_logits(c, llr)
bce = tf.reduce_mean(bce)
rate = tf.constant(1.0, tf.float32) - bce/tf.math.log(2.)
print(f"Rate: {rate:.2E} bit")

In [179]:
ris.phase_profile.values = tf.zeros_like(ris.phase_profile.values)

In [180]:
paths = scene.compute_paths(max_depth=5,
                            num_samples=1e6)

a, tau = paths.cir()
# Compute frequencies of subcarriers and center around carrier frequency
frequencies = subcarrier_frequencies(fft_size, subcarrier_spacing)

# Compute the frequency response of the channel at frequencies.
h_freq = cir_to_ofdm_channel(frequencies,
                             a,
                             tau,
                             normalize=True)

In [ ]:
num_training_iterations = 300 # Number of training iterations
training = True
if training:
    model = E2ESystem('neural-receiver', training=True)

    optimizer = tf.keras.optimizers.Adam()

    for i in range(num_training_iterations):
        # Sampling a batch of SNRs
        ebno_db = tf.random.uniform(shape=[], minval=ebno_db_min, maxval=ebno_db_max)
        # Forward pass
        with tf.GradientTape() as tape:
            rate = model(training_batch_size, ebno_db)
            loss = - rate
            print(loss.numpy())

        # Computing and applying gradients
        weights = model.trainable_weights
        grads = tape.gradient(loss, weights)
        optimizer.apply_gradients(zip(grads, weights))
        if i % 100 == 0:
            print('Iteration {}/{}  Rate: {:.4f} bit'.format(i, num_training_iterations, rate.numpy()), end='\r')



In [ ]:
def save_weights(model, model_weights_path):
    weights = model.get_weights()
    with open(model_weights_path, 'wb') as f:
        pickle.dump(weights, f)
save_weights(model, model_weights_path)
# Range of SNRs over which the systems are evaluated
ebno_dbs = np.arange(ebno_db_min, # Min SNR for evaluation
                     ebno_db_max, # Max SNR for evaluation
                     1) # Step
# function to load and set weights of a model
def load_weights(model, model_weights_path):
    model(1, tf.constant(10.0, tf.float32))
    with open(model_weights_path, 'rb') as f:
        weights = pickle.load(f)
    model.set_weights(weights)

model = E2ESystem('baseline-perfect-csi')
ber,_ = sim_ber(model, ebno_dbs, batch_size=64, num_target_block_errors=100, max_mc_iter=100)
BER['baseline-perfect-csi'] = ber.numpy()

model = E2ESystem('baseline-ls-estimation')
ber,_ = sim_ber(model, ebno_dbs, batch_size=64, num_target_block_errors=100, max_mc_iter=100)
BER['baseline-ls-estimation'] = ber.numpy()

model = E2ESystem('neural-receiver')

# Run one inference to build the layers and loading the weights
model(1, tf.constant(10.0, tf.float32))
with open(model_weights_path, 'rb') as f:
    weights = pickle.load(f)
model.set_weights(weights)

# Evaluations
ber,_ = sim_ber(model, ebno_dbs, batch_size=64, num_target_block_errors=100, max_mc_iter=100)
BER['neural-receiver'] = ber.numpy()

In [ ]:
binary_source = BinarySource()
mapper = Mapper("qam", num_bits_per_symbol)
rg_mapper = ResourceGridMapper(resource_grid)
#neural_receiver = model_conventional._neural_receiver
rg_demapper = ResourceGridDemapper(resource_grid, stream_manager)
channel = ApplyOFDMChannel(add_awgn=True)
batch_size = 64
ebno_db = tf.fill([batch_size], 5.0)
no = ebnodb2no(ebno_db, num_bits_per_symbol, coderate)
## Transmitter
# Generate codewords
c = binary_source([batch_size, 1, 1, n])
print("c shape: ", c.shape)
# Map bits to QAM symbols
x = mapper(c)
print("x shape: ", x.shape)
# Map the QAM symbols to a resource grid
x_rg = rg_mapper(x)
print("x_rg shape: ", x_rg.shape)
print("h_freq shape: ", h_freq.shape)
######################################
## Channel
no_ = expand_to_rank(no, tf.rank(x_rg))
print("no shape: ", no.shape)
print("no_ shape: ", no_.shape)
# Apply channel
channel = ApplyOFDMChannel(add_awgn=True)
y = channel([x_rg, h_freq, no])
print("y shape: ", y.shape)
######################################
#_lmmse_equ = LMMSEEqualizer(resource_grid, stream_manager, )
##_removed_null_subc = RemoveNulledSubcarriers(resource_grid)
######################################
## Receiver
_lmmse_equ = LMMSEEqualizer(resource_grid, stream_manager)
_demapper = Demapper("app", "qam", num_bits_per_symbol)
_ls_est = LSChannelEstimator(resource_grid, interpolation_type="nn")
# The neural receiver computes LLRs from the frequency domain received symbols and N0
#y = tf.squeeze(y, axis=1)
#h_hat = _removed_null_subc(h_freq) # Extract non-null subcarriers
#batch_size = tf.shape(y)[0]  # Or pass batch_size explicitly if available
h_hat, err_var = _ls_est([y, no])
# Tile h_hat across the batch dimension
#h_hat = tf.tile(h_hat, [batch_size, 1, 1, 1, 1, 14, 1])
x_hat, no_eff = _lmmse_equ([y, h_hat, err_var, no]) # LMMSE equalization
no_eff_= expand_to_rank(no_eff, tf.rank(x_hat))
llr = _demapper([x_hat, no_eff_]) # Demapping

bce = tf.nn.sigmoid_cross_entropy_with_logits(c, llr)
bce = tf.reduce_mean(bce)
rate = tf.constant(1.0, tf.float32) - bce/tf.math.log(2.)
print(f"Rate: {rate:.2E} bit")

In [ ]:
plt.figure(figsize=(10,6))
# Baseline - Perfect CSI
plt.semilogy(ebno_dbs, BER['baseline-perfect-csi'], 'o-', c=f'C0', label=f'Baseline - Perfect CSI')
# Baseline - LS Estimation
plt.semilogy(ebno_dbs, BER['baseline-ls-estimation'], 'x--', c=f'C1', label=f'Baseline - LS Estimation')
# Neural receiver
plt.semilogy(ebno_dbs, BER['neural-receiver'], 's-.', c=f'C2', label=f'Neural receiver')

plt.semilogy(ebno_dbs, BER['baseline-perfect-csi-OpRIS'], 'o-', c=f'C6', label=f'baseline-perfect-csi-OpRIS')
# Baseline - LS Estimation
plt.semilogy(ebno_dbs, BER['baseline-ls-estimation-OpRIS'], 'x--', c=f'C4', label=f'baseline-ls-estimation-OpRIS')
# Neural receiver
plt.semilogy(ebno_dbs, BER['neural-receiver-OpRIS'], 's-.', c=f'C5', label=f'neural-receiver-OpRIS')
#
plt.xlabel(r"$E_b/N_0$ (dB)")
plt.ylabel("BER")
plt.grid(which="both")
plt.ylim((1e-4, 1.0))
plt.legend()
plt.tight_layout()